## ALS

## 0. Setup

In [1]:
%load_ext autotime

time: 197 µs (started: 2024-03-27 21:23:15 -03:00)


In [2]:
#---- Bibliotecas:

# Análise e manipulação de dados:

import pandas as pd
import swifter
import numpy as np

# Visualização de dados:

import plotly.io as pio
import plotly.express as px

# Manuseio nos dados

from os import chdir

# ALS

from implicit import als

time: 1.39 s (started: 2024-03-27 21:23:15 -03:00)


/home/rafa/Documentos/github/book-recommendation/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#---- Mudando o diretório raiz para a pasta principal do projeto

chdir('../../')

#---- Template dos gráficos:

pio.templates.default = "plotly_white"

#---- Carregando as funções

from functions import *

time: 222 ms (started: 2024-03-27 21:23:17 -03:00)


## 1. Extração dos dados

In [4]:
#---- Dados de treino

train_data = load_sparse_csr('03-data/02-processed/04-ALS-train-data.npz')

train_data

<22569x158236 sparse matrix of type '<class 'numpy.int64'>'
	with 346011 stored elements in Compressed Sparse Row format>

time: 24.1 ms (started: 2024-03-27 21:23:17 -03:00)


In [5]:
#---- Dados de teste

df_test = pd.read_parquet('03-data/02-processed/02-test_data.parquet')

df_test.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country,RN
0,276747,1885408226,7,The Golden Rule of Schmoozing,Aye Jaye,1998,Listen &amp; Live Audio,25.0,iowa city,iowa,usa,5
1,276762,N3453124715,4,None,None,None,None,25.0,duisburg,nordrhein-westfalen,germany,4
2,276772,3596151465,10,Henry der Held.,Roddy Doyle,2001,"Fischer (Tb.), Frankfurt",NaN,bonn,nordrhein-westfalen,germany,3
3,276786,8478442588,6,El Elogio de La Sombra,Tanazaki,1998,Siruela,34.0,madrid,madrid,spain,4
4,276788,055310666X,10,False Memory,Dean R. Koontz,1999,Bantam Books,NaN,mentone,california,usa,3


time: 72.9 ms (started: 2024-03-27 21:23:17 -03:00)


In [6]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22568 entries, 0 to 22567
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   User-ID              22568 non-null  int64  
 1   ISBN                 22568 non-null  object 
 2   Book-Rating          22568 non-null  int64  
 3   Book-Title           18886 non-null  object 
 4   Book-Author          18886 non-null  object 
 5   Year-Of-Publication  18886 non-null  object 
 6   Publisher            18886 non-null  object 
 7   Age                  15357 non-null  float64
 8   city                 22568 non-null  object 
 9   state                22568 non-null  object 
 10  country              22568 non-null  object 
 11  RN                   22568 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 2.1+ MB
time: 20.5 ms (started: 2024-03-27 21:23:17 -03:00)


In [7]:
#---- Dados de treino

df_train = pd.read_parquet('03-data/02-processed/05-ALS-df-train-data.parquet')

df_train.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country,RN,USER_ID_CAT,IBSN_ID_CAT,USER_ID_INT,ISBN_ID_INT
0,276747,0060517794,9,Little Altars Everywhere,Rebecca Wells,2003,HarperTorch,25.0,iowa city,iowa,usa,1,22404,3844,1,1
1,276747,0671537458,9,Waiting to Exhale,Terry McMillan,1995,Pocket,25.0,iowa city,iowa,usa,2,22404,78067,1,2
2,276747,0679776818,8,Birdsong: A Novel of Love and War,Sebastian Faulks,1997,Vintage Books USA,25.0,iowa city,iowa,usa,3,22404,83361,1,3
3,276747,0943066433,7,How to Deal With Difficult People,Rick Brinkman,1995,Careertrack Inc.,25.0,iowa city,iowa,usa,4,22404,118092,1,4
4,276762,0380711524,5,See Jane Run,Joy Fielding,1992,Avon,25.0,duisburg,nordrhein-westfalen,germany,1,22405,37575,2,5


time: 513 ms (started: 2024-03-27 21:23:17 -03:00)


In [8]:
# #---- De-para's:

# de_para_isbn = pd.read_parquet('03-data/02-processed/05-ALS-DE-PARA-ISBN.parquet')
# de_para_user = pd.read_parquet('03-data/02-processed/06-ALS-DE-PARA-USER.parquet')

time: 269 µs (started: 2024-03-27 21:23:18 -03:00)


In [9]:
# #---- Full data:

# df_full = pd.read_parquet('03-data/02-processed/03-full_data.parquet')

# df_full.head()

time: 7.43 ms (started: 2024-03-27 21:23:18 -03:00)


## 2. Modelagem

In [10]:
model = als.AlternatingLeastSquares(factors = 30, random_state = 42)

model

/home/rafa/Documentos/github/book-recommendation/venv/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


time: 28.6 ms (started: 2024-03-27 21:23:18 -03:00)


In [11]:
model.fit(train_data)

100%|███████████████████████████████████████████| 15/15 [01:59<00:00,  7.94s/it]

time: 1min 59s (started: 2024-03-27 21:23:18 -03:00)


In [13]:
#---- DE-PARA USER_ID e ISBN:

de_para_user_id = dict(zip(df_train['User-ID'].tolist(), df_train['USER_ID_INT'].tolist()))
de_para_isbn = dict(zip(df_train['ISBN'].tolist(), df_train['ISBN_ID_INT'].tolist()))

time: 192 ms (started: 2024-03-27 21:25:17 -03:00)


## 3. Recomendações

In [31]:
user_id_rec = 276747

de_para_user_id_rec = de_para_user_id[user_id_rec]

de_para_user_id_rec

1

time: 3.13 ms (started: 2024-03-27 21:50:28 -03:00)


In [14]:
# #---- TOP 10 livros mais bem avaliados pelo usuário

# df_train\
#     .query(f'`User-ID` == {user_id_rec}')\
#     .query(f'`Book-Rating` != 0')\
#     .sort_values(by = 'Book-Rating', ascending = False)[['ISBN', 'Book-Title', 'Book-Rating']]\
#     .drop_duplicates()\
#     .head(10)

,ISBN,Book-Title,Book-Rating
221525,0439064872,Harry Potter and the Chamber of Secrets (Book 2),10
221526,0439136369,Harry Potter and the Prisoner of Azkaban (Book 3),10
221527,0439139597,Harry Potter and the Goblet of Fire (Book 4),10
221528,043935806X,Harry Potter and the Order of the Phoenix (Boo...,10
221529,043936213X,Harry Potter and the Sorcerer's Stone (Book 1),10
221524,0140386335,The Neverending Story,9


time: 27.3 ms (started: 2024-03-27 21:22:22 -03:00)


In [15]:
# #---- Lista de ISBN que o usuário já leu

# readed_isbn_by_user = df_train\
#     .query(f'`User-ID` == {user_id_rec}')['Book-Title']\
#     .unique()\
#     .tolist()

# readed_isbn_by_user

['The Neverending Story',
 'Harry Potter and the Chamber of Secrets (Book 2)',
 'Harry Potter and the Prisoner of Azkaban (Book 3)',
 'Harry Potter and the Goblet of Fire (Book 4)',
 'Harry Potter and the Order of the Phoenix (Book 5)',
 "Harry Potter and the Sorcerer's Stone (Book 1)"]

time: 10.6 ms (started: 2024-03-27 21:22:22 -03:00)


In [16]:
# not_recommended_list_to_user = df_train\
#     .query(f'`Book-Title`.isin({readed_isbn_by_user})')['ISBN_ID_INT']\
#     .unique()\
#     .tolist()

time: 57.3 ms (started: 2024-03-27 21:22:22 -03:00)


In [17]:
# #---- Top10 recomendações para o usuário

# recommendations = model.recommend(de_para_user_id_rec,
#                                   train_data[de_para_user_id_rec], 
#                                   filter_already_liked_items = True,
#                                   N = 10,
#                                   filter_items = not_recommended_list_to_user)

# print(f'Recomendações: {recommendations}')

# list_recommendations = recommendations[0].tolist()

# #---- De-para das recomendações para ISBN

# # isbn_recommendations = df_train\
# #     .query(f'ISBN_ID_INT.isin({list_recommendations})')['ISBN']\
# #     .unique()\
# #     .tolist()

# isbn_recommendations = list({k: v for k, v in de_para_isbn.items() if v in list_recommendations}.keys())

# df_train\
#     .pipe(filter_list_ordered, 
#           col = 'ISBN', 
#           list_filter = isbn_recommendations)\
#     [['ISBN', 'Book-Title']]\
#     .drop_duplicates()['Book-Title'].unique()

Recomendações: (array([ 1623,   810,  1478,  2081,  2077, 19063,  2529,   200,  1316,
         305], dtype=int32), array([0.6973636 , 0.3413098 , 0.31499463, 0.2582363 , 0.24672447,
       0.24356365, 0.24171497, 0.23169836, 0.2124185 , 0.20761147],
      dtype=float32))


array(['To Kill a Mockingbird', 'Fahrenheit 451',
       'The Hobbit : The Enchanting Prelude to The Lord of the Rings',
       'Angels &amp; Demons',
       'The Fellowship of the Ring (The Lord of the Rings, Part 1)',
       "Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))",
       'The Catcher in the Rye',
       'The Two Towers (The Lord of the Rings, Part 2)',
       'The Bad Beginning (A Series of Unfortunate Events, Book 1)',
       'The Return of the King (The Lord of the Rings, Part 3)'],
      dtype=object)

time: 415 ms (started: 2024-03-27 21:22:22 -03:00)


## 4. Avaliação das métricas

In [18]:
# Criar uma função que olhe a coluna User-ID e faça as TOP N recomendações. Resultado: lista
# Validar se dentro dessa lista de recomendações, está o livro-teste
# Criaremos uma flag: se o livro está dentro das recomendações, 1, senão 0
# Proporção dos valores 1 (Precision @ K)

time: 348 µs (started: 2024-03-27 21:22:23 -03:00)


In [32]:
#---- Criando uma função para recomendar K itens a partir do nome do usuário

def choose_books_als(user_id_to_rec: str, dict_de_para_user: dict, dict_de_para_isbn: dict, rec_matrix, n: int = 10):

    # User_id para fazer a recomendação:

    user_id_de_para = dict_de_para_user[user_id_to_rec]

    recommendations = model.recommend(user_id_de_para,
                                      rec_matrix[user_id_de_para], 
                                      filter_already_liked_items = True,
                                      N = n)

    list_recommendations = recommendations[0].tolist()

    #---- De-para das recomendações para ISBN

    isbn_recommendations = list({k: v for k, v in de_para_isbn.items() if v in list_recommendations}.keys())

    return isbn_recommendations

time: 873 µs (started: 2024-03-27 21:50:48 -03:00)


In [33]:
choose_books_als(user_id_to_rec = 276747,
                 dict_de_para_user = de_para_user_id,
                 dict_de_para_isbn = de_para_isbn,
                 rec_matrix = train_data)

1


['0446672211',
 '0385492081',
 '0804106304',
 '0440226430',
 '0671003755',
 '067976402X',
 '0060930535',
 '044021145X',
 '0375707972',
 '068484477X']

time: 41.3 ms (started: 2024-03-27 21:50:49 -03:00)


In [21]:
# df_test_copy = df_test\
#     .sample(1000)

time: 22.7 ms (started: 2024-03-27 21:22:23 -03:00)


In [15]:
df_test['TOP_N_RECOMMENDATIONS'] = df_test\
    .swifter.apply(lambda x: choose_books_als(user_id_to_rec = x['User-ID'], 
                                      dict_de_para_user = de_para_user_id,
                                      dict_de_para_isbn = de_para_isbn,
                                      rec_matrix = train_data), axis = 1)

Pandas Apply: 100%|███████████████████████| 22568/22568 [14:55<00:00, 25.19it/s]

time: 16min 38s (started: 2024-03-27 21:25:17 -03:00)


In [17]:
df_test\
    .head(5)

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country,RN,TOP_N_RECOMMENDATIONS
0,276747,1885408226,7,The Golden Rule of Schmoozing,Aye Jaye,1998,Listen &amp; Live Audio,25.0,iowa city,iowa,usa,5,"[0446672211, 0385492081, 0804106304, 044022643..."
1,276762,N3453124715,4,None,None,None,None,25.0,duisburg,nordrhein-westfalen,germany,4,"[0449221512, 0446610038, 0425163407, 044023722..."
2,276772,3596151465,10,Henry der Held.,Roddy Doyle,2001,"Fischer (Tb.), Frankfurt",NaN,bonn,nordrhein-westfalen,germany,3,"[0061009059, 0811811409, 1400031354, 067100179..."
3,276786,8478442588,6,El Elogio de La Sombra,Tanazaki,1998,Siruela,34.0,madrid,madrid,spain,4,"[2266104535, 3257227809, 3404148665, 087352379..."
4,276788,055310666X,10,False Memory,Dean R. Koontz,1999,Bantam Books,NaN,mentone,california,usa,3,"[0345339681, 0439064872, 0439136369, 043913960..."


time: 29.4 ms (started: 2024-03-27 21:42:40 -03:00)


## 5. Validando o Precision @  K

In [21]:
def calculate_precision_k_als(df: pd.DataFrame, col_books: str):

    df['flag_recommendations'] = df.apply(lambda x: x[col_books] in x['TOP_N_RECOMMENDATIONS'], axis = 1)

    value_precision_k = df.query('flag_recommendations == 1').shape[0] / df.shape[0] * 100

    precision_k = 0 if value_precision_k == 0 else value_precision_k

    return df, precision_k

time: 1.77 ms (started: 2024-03-27 21:46:21 -03:00)


In [24]:
calculate_precision_k_als(df = df_test, col_books = 'Book-Title')[0]

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Age,city,state,country,RN,TOP_N_RECOMMENDATIONS,flag_recommendations
0,276747,1885408226,7,The Golden Rule of Schmoozing,Aye Jaye,1998,Listen &amp; Live Audio,25.0,iowa city,iowa,usa,5,"[0446672211, 0385492081, 0804106304, 044022643...",False
1,276762,N3453124715,4,None,None,None,None,25.0,duisburg,nordrhein-westfalen,germany,4,"[0449221512, 0446610038, 0425163407, 044023722...",False
2,276772,3596151465,10,Henry der Held.,Roddy Doyle,2001,"Fischer (Tb.), Frankfurt",NaN,bonn,nordrhein-westfalen,germany,3,"[0061009059, 0811811409, 1400031354, 067100179...",False
3,276786,8478442588,6,El Elogio de La Sombra,Tanazaki,1998,Siruela,34.0,madrid,madrid,spain,4,"[2266104535, 3257227809, 3404148665, 087352379...",False
4,276788,055310666X,10,False Memory,Dean R. Koontz,1999,Bantam Books,NaN,mentone,california,usa,3,"[0345339681, 0439064872, 0439136369, 043913960...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22563,276680,1931561648,9,The Time Traveler's Wife (Today Show Book Club...,Audrey Niffenegger,2003,MacAdam/Cage Publishing,55.0,hopewell junction,new york,usa,91,"[0452282152, 0446672211, 0060934417, 067102736...",False
22564,276681,1931333246,10,Chorus,Kurt Messick,2003,Dry Bones Press,43.0,chicago,illinois,usa,11,"[0385504209, 0312195516, 0399501487, 031666634...",False
22565,276683,0553278231,9,Cannery Row,John Steinbeck,1986,Bantam Books,NaN,charlotte,north carolina,usa,3,"[0345337662, 0345350499, 0380789035, 034531386...",False
22566,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,NaN,fort lee,new jersey,usa,29,"[0061009059, 0553280368, 0312980140, 055328034...",False


time: 288 ms (started: 2024-03-27 21:47:21 -03:00)


# Para a próxima rodada de estudo:

- Otimizar o código
- Continuar a avaliação da métrica

# TO-DO's

- Ler um pouco mais sobre a teoria do ALS
- Criar um código que exclua o primeiro item da recomendação do usuário

# Validações

In [ ]:
df_train['FLAG_HP'] = np.where(df_train['Book-Title'].str.contains('Potter'), 1, 0)

In [ ]:
test = df_train[['User-ID', 'FLAG_HP']]\
    .value_counts()\
    .reset_index()\
    .sort_values(by = ['User-ID', 'FLAG_HP'])

test['percent'] = test['count'] / test.groupby('User-ID')['count'].transform('sum') * 100

test\
    .sort_values(by = 'percent', ascending = False)\
    .query('FLAG_HP == 1 & count >= 3')\
    .head(20)

In [ ]:
df_train\
    .query('`User-ID` == 252829')\
    .sort_values(by = 'Book-Title')